<a href="https://colab.research.google.com/github/7ryean/bert/blob/master/22_02_20bert%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools

!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null

!apt-get update -qq 2>&1 > /dev/null

!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth

auth.authenticate_user()

from oauth2client.client import GoogleCredentials

creds = GoogleCredentials.get_application_default()

import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL

vcode = getpass.getpass()

!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 155113 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.27-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.27-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.27-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [2]:
!mkdir -p Gdrive

In [3]:
!google-drive-ocamlfuse Gdrive

In [4]:
!ls

adc.json  Gdrive  sample_data


In [5]:
!ls Gdrive/bert_files

modeling.py  optimization.py  __pycache__  run_squad.py  tokenization.py


In [6]:
import tensorflow as tf
import numpy as np
import os

try:
  device_name = os.environ['COLAB_TPU_ADDR']
  TPU_ADDRESS = 'grpc://' + device_name
  print('Found TPU at: {}'.format(TPU_ADDRESS))

except KeyError:
  print('TPU not found')

Found TPU at: grpc://10.102.250.66:8470


In [7]:
!python3.7 -m pip install tensorflow-cpu==1.15.0 

     |████████████████████████████████| 110.8 MB 2.0 kB/s 
     |████████████████████████████████| 503 kB 40.4 MB/s 
     |████████████████████████████████| 50 kB 5.5 MB/s 
     |████████████████████████████████| 3.8 MB 53.9 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=6e06d96d5682835cfad8c9224d62009da768770bbf8b6dbeec0d68f49a8f57df
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Succe

In [16]:
!python Gdrive/bert_files/run_squad.py --vocab_file=gs://first_test_bert/pretrained_files/vocab.txt --bert_config_file=gs://first_test_bert/pretrained_files/bert_config.json --init_checkpoint=gs://first_test_bert/pretrained_files/bert_model.ckpt.data-00000-of-00001 --do_train=True --train_file=Gdrive/korquad_files/korquad2.1_train_01.json --do_predict=True --predict_file=Gdrive/korquad_files/korquad2.1_dev_01.json --train_batch_size=16 --learning_rate=3e-5 --num_train_epochs=2.0 --max_seq_length=384 --doc_stride=128 --output_dir=Gdrive/temp --use_tpu=True --do_lower_case=False --tpu_name=grpc://10.102.250.66:8470 \




W0219 15:20:46.065962 140193626285952 module_wrapper.py:139] From Gdrive/bert_files/run_squad.py:1127: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0219 15:20:46.066168 140193626285952 module_wrapper.py:139] From Gdrive/bert_files/run_squad.py:1127: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0219 15:20:46.066325 140193626285952 module_wrapper.py:139] From /content/Gdrive/bert_files/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0219 15:20:47.214761 140193626285952 module_wrapper.py:139] From Gdrive/bert_files/run_squad.py:1133: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorf

In [17]:
!python Gdrive/korquad_files/evaluate-2.0.py Gdrive/korquad_files/korquad2.1_dev_01.json Gdrive/temp/predictions.json

Traceback (most recent call last):
  File "Gdrive/korquad_files/evaluate-2.0.py", line 107, in <module>
    file_names = os.listdir(args.dataset_file)
NotADirectoryError: [Errno 20] Not a directory: 'Gdrive/korquad_files/korquad2.1_dev_01.json'


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!gcloud config set project poetic-orb-341813

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey



In [ ]:
import os

os.chdir("Gdrive")

In [ ]:
!gsutil cp gs://first_test_bert/temp/predictions.json /temp/predictions.json

CommandException: No URLs matched: gs://first_test_bert/temp/predictions.json


In [ ]:
!python korquad_files/evaluate-v1.0.py KorQuAD_v1.0_dev.json temp/predictions.json